# Code Explanation

- 천만영화와와 비천만영화를 비교하는데 있어, 동기간 개봉되었던 영화의 관람객 수는 큰 영향을 미칠 수 있는 요소이다.
- 김도현 팀원이 1위부터 58위까지 개별 영화들의 상영기간 정보가 담긴 파일을 총 58개 크롤링 해왔음.
- 개봉일 이전의 일자를 삭제해주고, 스크린점유율이 1%미만인 날의 일자도 삭제하여 총 상영기간이 산출되었음. 
- Selenium을 이용해 이 기간동안 동시상영된 영화들의 리스트를 뽑아 사전(json)으로 도출하였음.

In [2]:
movie_info_final=pd.read_csv("movie_info_final.csv")

In [3]:
movie_info_final

,Title,Release,Genre,Age,Nation,Type,Director,Main_actor,Audi
0,명량,2014-07-30,"사극,액션",15세이상관람가,한국,"필름,2D,4D",김한민,"최민식,류승룡,조진웅",17615919
1,극한직업,2019-01-23,코미디,15세이상관람가,한국,2D,이병헌,"류승룡,이하늬,진선규,이동휘,공명",16266338
2,신과함께-죄와 벌,2017-12-20,"판타지,드라마",12세이상관람가,한국,"2D,4D",김용화,"하정우,차태현,주지훈,김향기,마동석",14414658
3,국제시장,2014-12-17,드라마,12세이상관람가,한국,"필름,2D",윤제균,"황정민,김윤진,오달수,정진영",14264478
4,어벤져스: 엔드게임,2019-04-24,"액션,SF",12세이상관람가,미국,"2D,3D,4D,IMAX","안소니 루소,조 루소","로버트 다우니 주니어,크리스 에반스,마크 러팔로,크리스 헴스워스,스칼렛 요한슨,제레...",13977602
...,...,...,...,...,...,...,...,...,...
197,해리 포터와 불의 잔,2005-12-01,"액션,판타지",12세관람가,미국,"필름,4D",마이크 뉴웰,"다니엘 래드클리프,루퍼트 그린트,엠마 왓슨,랄프 파인즈",3499257
198,비긴 어게인,2014-08-13,멜로/로맨스,15세이상관람가,미국,"필름,2D",존 카니,"키이라 나이틀리,마크 러팔로,애덤 리바인,헤일리 스테인펠드,제임스 코든,캐서린 키너",3477672
199,태풍,2005-12-14,"드라마,액션",15세관람가,한국,"필름,2D",곽경택,"이정재,이미연,장동건",3471150
200,킹콩,2005-12-14,"드라마,어드벤처",15세관람가,미국,필름,피터 잭슨,"잭 블랙,애드리언 브로디,나오미 왓츠",3470059


## 사용한 모듈

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager    # 매 번 크롬 드라이버를 설치할 필요없이 자동으로
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.alert import Alert
from tqdm.notebook import tqdm
from urllib.request import urlopen
import time
import requests
import os
import glob
import shutil
import json

# 1. 파일 불러오기 - 영화가 상영되었던 기간의 확인을 위한 파일 불러오기

- 천만영화는 총 29개에 해당되므로, 2배에 해당되는 1위~58위까지의 영화를 확인하겠다.
- 이를 위해 개별 영화 파일들이 저장된 폴더의 목록을 확인한다.
- 파일의 경로는 김도현 팀원의 깃 경로이다.

In [2]:
csv_files = glob.glob("C:\\GitTest\\10M_Movie\\movie_DB\\individual_trend\\*.csv")   # glob 모듈로 폴더 내 파일 확인

file_name = []   # 빈 리스트 생성.

for file_path in csv_files:                            # 폴더 내 파일들의 이름을 불러와 리스트에 추가
    file_name.append(os.path.basename(file_path))


file_name          # file_name 리스트의 순서가 순위 별이 아닌 ㄱ,ㄴ,ㄷ 오름차순으로 정렬되어 있음.

['1987.csv',
 '7번방의 선물.csv',
 '검사외전.csv',
 '겨울왕국 2.csv',
 '겨울왕국.csv',
 '공조.csv',
 '과속스캔들.csv',
 '관상.csv',
 '광해, 왕이 된 남자.csv',
 '괴물.csv',
 '국가대표.csv',
 '국제시장.csv',
 '극한직업.csv',
 '기생충.csv',
 '도둑들.csv',
 '디워.csv',
 '명량.csv',
 '미션임파서블-고스트프로토콜.csv',
 '밀정.csv',
 '백두산.csv',
 '범죄도시2.csv',
 '베를린.csv',
 '베테랑.csv',
 '변호인.csv',
 '보헤미안 랩소디.csv',
 '부산행.csv',
 '설국열차.csv',
 '수상한 그녀.csv',
 '스파이더맨- 노 웨이 홈.csv',
 '스파이더맨- 파 프롬 홈.csv',
 '스파이더맨- 홈 커밍.csv',
 '신과함께-인과 연.csv',
 '신과함께-죄와 벌.csv',
 '실미도.csv',
 '써니.csv',
 '아바타- 물의 길.csv',
 '아바타.csv',
 '아이언맨 3.csv',
 '알라딘.csv',
 '암살.csv',
 '어벤져스- 에이지 오브 울트론.csv',
 '어벤져스- 엔드게임.csv',
 '어벤져스- 인피니티 워.csv',
 '엑시트.csv',
 '왕의 남자.csv',
 '인터스텔라.csv',
 '최종병기 활.csv',
 '캡틴 아메리카- 시빌 워.csv',
 '탑건- 매버릭.csv',
 '태극기 휘날리며.csv',
 '택시운전사.csv',
 '트랜스포머 3.csv',
 '트랜스포머- 패자의 역습.csv',
 '트랜스포머.csv',
 '한산- 용의 출현.csv',
 '해운대.csv',
 '해적- 바다로 간 산적.csv',
 '히말라야.csv']

In [3]:
len(file_name)        # 58개 잘 불러져 왔음

58

# 검증을 위해 비교할 파일 불러오기

- 1위부터 200위까지의 영화정보가 담긴 csv파일은 고은경 팀원의 깃 경로를 이용한다.

In [3]:
# 1위부터 200위까지 영화 이름, 개봉일, 감독 정보, 장르, 관람 가능 연령, 상영 타입등의 정보가 담긴 csv파일 불러오기
movie_rank = pd.read_csv("C:\\GitTest\\10M_Movie\\movie_DB\\individual_movie\\movie_info_final.csv")    

In [5]:
# 파일의 이름을 영화 제목으로 저장했기 때문에 ':' 과 같은 특수기호가 있으면 저장이 되지 않아, ':'를 '-' 로 바꿔줬었음. 
# 하지만, movie_rank 데이터에는 영화제목이 여전히 ":"로 표시되어 있기 때문에 이를 - 로 바꿔준다.
for i in range(58):
    movie_rank.iloc[i,0]=movie_rank.iloc[i,0].replace(":", "-")   

In [7]:
movie_rank

,영화,개봉일,장르,영상등급,제작국가,스크린타입,감독,주연,누적관객수
0,명량,2014-07-30,"사극,액션",15세이상관람가,한국,"필름,2D,4D",김한민,"최민식,류승룡,조진웅",17615919
1,극한직업,2019-01-23,코미디,15세이상관람가,한국,2D,이병헌,"류승룡,이하늬,진선규,이동휘,공명",16266338
2,신과함께-죄와 벌,2017-12-20,"판타지,드라마",12세이상관람가,한국,"2D,4D",김용화,"하정우,차태현,주지훈,김향기,마동석",14414658
3,국제시장,2014-12-17,드라마,12세이상관람가,한국,"필름,2D",윤제균,"황정민,김윤진,오달수,정진영",14264478
4,어벤져스- 엔드게임,2019-04-24,"액션,SF",12세이상관람가,미국,"2D,3D,4D,IMAX","안소니 루소,조 루소","로버트 다우니 주니어,크리스 에반스,마크 러팔로,크리스 헴스워스,스칼렛 요한슨,제레...",13977602
...,...,...,...,...,...,...,...,...,...
195,신의 한 수,2014-07-03,"범죄,액션",청소년관람불가,한국,"필름,2D",조범구,"정우성,안성기,이범수,김인권,이시영,안길강,최진혁",3566852
196,코코,2018-01-11,애니메이션,전체관람가,미국,"2D,4D,IMAX",리 언크리치,"가엘 가르시아 베르날,앤서니 곤잘레스,벤자민 브랫",3516880
197,해리 포터와 불의 잔,2005-12-01,"액션,판타지",12세이상관람가,미국,"필름,4D",마이크 뉴웰,"다니엘 래드클리프,루퍼트 그린트,엠마 왓슨,랄프 파인즈",3499257
198,비긴 어게인,2014-08-13,멜로/로맨스,15세이상관람가,미국,"필름,2D",존 카니,"키이라 나이틀리,마크 러팔로,애덤 리바인,헤일리 스테인펠드,제임스 코든,캐서린 키너",3477672


In [10]:
name = list(movie_rank['Title'][:58])            # 잘 바뀌어있음.
len(name)

28

# 2. 전처리 

- file_name 리스트의 순서가 순위 별이 아닌 ㄱ,ㄴ,ㄷ 오름차순으로 정렬되어 있기 때문에 pd.read_csv()를 하려면 순위로 된 리스트를 만들어 주어야 함

In [84]:
name_list = []
for i in name:
    name_list.append(i+".csv")
    
name_list                        # 리스트 내 파일명이 영화 순위별로 정렬 되었음

['명량.csv',
 '극한직업.csv',
 '신과함께-죄와 벌.csv',
 '국제시장.csv',
 '어벤져스: 엔드게임.csv',
 '겨울왕국 2.csv',
 '베테랑.csv',
 '아바타.csv',
 '도둑들.csv',
 '7번방의 선물.csv',
 '알라딘.csv',
 '암살.csv',
 '범죄도시2.csv',
 '광해, 왕이 된 남자.csv',
 '신과함께-인과 연.csv',
 '택시운전사.csv',
 '태극기 휘날리며.csv',
 '부산행.csv',
 '변호인.csv',
 '해운대.csv',
 '어벤져스: 인피니티 워.csv',
 '실미도.csv',
 '괴물.csv',
 '아바타: 물의 길.csv',
 '왕의 남자.csv',
 '어벤져스: 에이지 오브 울트론.csv',
 '인터스텔라.csv',
 '기생충.csv',
 '겨울왕국.csv',
 '보헤미안 랩소디.csv',
 '검사외전.csv',
 '엑시트.csv',
 '설국열차.csv',
 '관상.csv',
 '아이언맨 3.csv',
 '캡틴 아메리카: 시빌 워.csv',
 '해적: 바다로 간 산적.csv',
 '수상한 그녀.csv',
 '백두산.csv',
 '과속스캔들.csv',
 '탑건: 매버릭.csv',
 '국가대표.csv',
 '스파이더맨: 파 프롬 홈.csv',
 '디워.csv',
 '공조.csv',
 '트랜스포머 3.csv',
 '히말라야.csv',
 '스파이더맨: 노 웨이 홈.csv',
 '미션임파서블:고스트프로토콜.csv',
 '밀정.csv',
 '최종병기 활.csv',
 '트랜스포머.csv',
 '트랜스포머: 패자의 역습.csv',
 '써니.csv',
 '한산: 용의 출현.csv',
 '스파이더맨: 홈 커밍.csv',
 '1987.csv',
 '베를린.csv']

In [85]:
len(name_list)

58

# Selenium 크롤링 
- 58개의 파일을 열고, selenium을 이용해 영화가 상영되었던 기간 동안 1위부터 10위까지 불러온 후, 해당 영화가 58위 안에 속하면, 결과에 추가해준다.

In [66]:
driver = webdriver.Chrome("chromedriver")
driver.maximize_window()                                  # 원활환 selenium의 실행을 위해 윈도우 창의 크기를 최대화 한다.
url = ('https://www.kobis.or.kr/kobis/business/stat/boxs/findPeriodBoxOfficeList.do')
driver.get(url)

result=[]          # 결과를 담아줄 리스트
 
for i in tqdm(range(len(name_list))):
    df = pd.read_csv("C:\\GitTest\\10M_Movie\\movie_DB\\individual_trend\\{}".format(name_list[i]), index_col=0)
    
    a = df.iloc[0]["날짜"]         # 첫번째 행의 날짜(조회기간에서 조회시작 시점)
    b = df.iloc[-1]["날짜"]        # 마지막 행의 날짜(조회기간에서 조회마무리 시점)
    
    # 조회 기간 From 클릭
    period_start = driver.find_element(By.ID, "sSearchFrom")
    period_start.click()
    
    # 조회 기간 From에 이미 적혀있는 내용 지우기
    period_start = driver.find_element(By.ID, "sSearchFrom")
    period_start.clear()

    # 날짜 형식을 올바르게 입력해주세요. 오류창 -> 무시하고 Enter
    da = Alert(driver)
    da.accept()
    
    # 새로운 값을 입력
    period_start = driver.find_element(By.ID, "sSearchFrom")
    period_start.send_keys(a)                                  # 영화가 상영되었던 날짜의 시작 시점.
    period_start.send_keys(Keys.ENTER)

    #######################################################################################    
        
    # 조회 기간 TO 클릭
    period_end = driver.find_element(By.ID, "sSearchTo")
    period_end.click()
    
    # 조회 기간 TO에 이미 적혀있는 내용 지우기
    period_end = driver.find_element(By.ID, "sSearchTo")
    period_end.clear()

    # 날짜 형식을 올바르게 입력해주세요 오류창 -> 무시하기
    da = Alert(driver)
    da.accept()
    
    
    # 새로운 값을 입력
    period_end.send_keys(b)                                     # 영화가 상영되었던 날짜의 마지막 시점.
    period_end.send_keys(Keys.ENTER)          
    
    # 조회버튼 클릭
    driver.find_element(By.CLASS_NAME, "btn_blue").click()

    # 조회버튼을 클릭 했을 때, 로딩중입니다 기다리기
    search_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "btn_blue")))
    
    
    
    # 조회버튼을 클릭하고, 설정된 조회 기간동안 상영되었던 영화들의 정보를 가져옴
    text = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[4]/table/tbody').text
    
    
    # 내용이 \로 분리되기 때문에 split 해 준 후, 1위~ 10위까지 이름만 가져오려면, 3개씩 건너 뛰면서 가져오면 됨
    text.split("\n")[1:29:3] 
    
    
    same_period = []                                          # 임시 리스트
    
    for element in text.split("\n")[1:29:3]:
        print(element)
        if element in list(movie_rank1["Title"].loc[0:58]):   # 조회된 기간 동안 10위까지의 영화가 역대 영화 순위 58위 안에 들면
            same_period.append(element)                       # 그 영화들을 임시 리스트에 넣어준 후,
            print(same_period)
            
    result.append(same_period)                                # 결과를 담아줄 리스트에 넣어준다.

  0%|          | 0/58 [00:00<?, ?it/s]

명량
['명량']
해적: 바다로 간 산적
['명량', '해적: 바다로 간 산적']
타짜-신의 손
비긴 어게인
인투 더 스톰
루시
메이즈 러너
드래곤 길들이기 2
두근두근 내 인생
해무(海霧)
극한직업
['극한직업']
캡틴 마블
증인
사바하
알리타: 배틀 엔젤
뺑반
돈
드래곤 길들이기 3
항거:유관순 이야기
극장판 헬로카봇: 옴파로스 섬의 비밀
신과함께-죄와 벌
['신과함께-죄와 벌']
1987
['신과함께-죄와 벌', '1987']
블랙 팬서
코코
그것만이 내 세상
강철비
메이즈 러너: 데스 큐어
조선명탐정: 흡혈괴마의 비밀
쥬만지: 새로운 세계
위대한 쇼맨
국제시장
['국제시장']
킹스맨 : 시크릿 에이전트
조선명탐정 : 사라진 놉의 딸
님아, 그 강을 건너지 마오
호빗: 다섯 군대 전투
빅 히어로
기술자들
강남 1970
테이큰 3
오늘의 연애
어벤져스: 엔드게임
['어벤져스: 엔드게임']
기생충
['어벤져스: 엔드게임', '기생충']
악인전
알라딘
['어벤져스: 엔드게임', '기생충', '알라딘']
걸캅스
나의 특별한 형제
뽀로로 극장판 보물섬 대모험
명탐정 피카츄
고질라: 킹 오브 몬스터
배심원들
겨울왕국 2
['겨울왕국 2']
백두산
['겨울왕국 2', '백두산']
시동
천문: 하늘에 묻는다
닥터 두리틀
포드 V 페라리
블랙머니
쥬만지: 넥스트 레벨
미드웨이
해치지않아
베테랑
['베테랑']
사도
암살
['베테랑', '암살']
앤트맨
미션 임파서블: 로그네이션
메이즈 러너: 스코치 트라이얼
탐정 : 더 비기닝
인턴
마션
뷰티 인사이드
아바타
['아바타']
전우치
의형제
하모니
셜록 홈즈
퍼시 잭슨과 번개 도둑
이상한 나라의 앨리스
용서는 없다
평행이론
주유소 습격사건 2
도둑들
['도둑들']
광해, 왕이 된 남자
['도둑들', '광해, 왕이 된 남자']
바람과 함께 사라지다
다크 나이트 라이즈
이웃사람
아이스 에이지 4: 대륙 이동설
공모자들
테이큰 2
새미의 어드벤쳐 2
토탈 리콜
7번방의 선물
['7번방의 선물']
베를린
['7번방의 선물', '베를린

## 결과

In [86]:
result                          # ex) 명량이 상영되었던 동기간에 상영된 영화들 중 58위 안에 있는 영화들 

[['명량', '해적: 바다로 간 산적'],
 ['극한직업'],
 ['신과함께-죄와 벌', '1987'],
 ['국제시장'],
 ['어벤져스: 엔드게임', '기생충', '알라딘'],
 ['겨울왕국 2', '백두산'],
 ['베테랑', '암살'],
 ['아바타'],
 ['도둑들', '광해, 왕이 된 남자'],
 ['7번방의 선물', '베를린'],
 ['알라딘', '기생충', '엑시트', '스파이더맨: 파 프롬 홈'],
 ['베테랑', '암살'],
 ['범죄도시2', '탑건: 매버릭'],
 ['광해, 왕이 된 남자'],
 ['신과함께-인과 연'],
 ['택시운전사'],
 ['태극기 휘날리며', '실미도'],
 ['부산행', '밀정'],
 ['변호인', '겨울왕국', '수상한 그녀'],
 ['해운대', '국가대표'],
 ['어벤져스: 인피니티 워'],
 ['태극기 휘날리며', '실미도'],
 ['괴물'],
 ['아바타: 물의 길'],
 ['왕의 남자'],
 ['어벤져스: 에이지 오브 울트론'],
 ['인터스텔라', '국제시장'],
 ['알라딘', '기생충', '스파이더맨: 파 프롬 홈'],
 ['겨울왕국', '수상한 그녀', '변호인'],
 ['보헤미안 랩소디', '극한직업'],
 ['검사외전'],
 ['엑시트'],
 ['관상', '설국열차'],
 ['관상'],
 ['아이언맨 3'],
 ['캡틴 아메리카: 시빌 워'],
 ['명량', '해적: 바다로 간 산적'],
 ['겨울왕국', '수상한 그녀'],
 ['백두산', '겨울왕국 2'],
 ['과속스캔들'],
 ['탑건: 매버릭', '한산: 용의 출현'],
 ['해운대', '국가대표'],
 ['스파이더맨: 파 프롬 홈', '알라딘', '엑시트', '기생충'],
 ['디워'],
 ['공조'],
 ['트랜스포머 3', '써니'],
 ['히말라야', '검사외전'],
 ['스파이더맨: 노 웨이 홈'],
 ['미션임파서블:고스트프로토콜'],
 ['밀정'],
 ['최종병기 활'],
 ['트랜스포머', '디워'],
 ['해운대',

In [90]:
name = list(movie_rank1['Title'][:58])                            
name                                     # 1위부터 58위까지의 영화 제목 리스트

['명량',
 '극한직업',
 '신과함께-죄와 벌',
 '국제시장',
 '어벤져스: 엔드게임',
 '겨울왕국 2',
 '베테랑',
 '아바타',
 '도둑들',
 '7번방의 선물',
 '알라딘',
 '암살',
 '범죄도시2',
 '광해, 왕이 된 남자',
 '신과함께-인과 연',
 '택시운전사',
 '태극기 휘날리며',
 '부산행',
 '변호인',
 '해운대',
 '어벤져스: 인피니티 워',
 '실미도',
 '괴물',
 '아바타: 물의 길',
 '왕의 남자',
 '어벤져스: 에이지 오브 울트론',
 '인터스텔라',
 '기생충',
 '겨울왕국',
 '보헤미안 랩소디',
 '검사외전',
 '엑시트',
 '설국열차',
 '관상',
 '아이언맨 3',
 '캡틴 아메리카: 시빌 워',
 '해적: 바다로 간 산적',
 '수상한 그녀',
 '백두산',
 '과속스캔들',
 '탑건: 매버릭',
 '국가대표',
 '스파이더맨: 파 프롬 홈',
 '디워',
 '공조',
 '트랜스포머 3',
 '히말라야',
 '스파이더맨: 노 웨이 홈',
 '미션임파서블:고스트프로토콜',
 '밀정',
 '최종병기 활',
 '트랜스포머',
 '트랜스포머: 패자의 역습',
 '써니',
 '한산: 용의 출현',
 '스파이더맨: 홈 커밍',
 '1987',
 '베를린']

In [91]:
same_period_info = dict(zip(name, result))             # 이들을 dict로 매핑 시켜준다.

#### 결과 dict

- ex) 명량이 상영되었을 당시 해적: 바다로 간 산적은 아쉽게 천만 영화를 달성하지 못함.

In [92]:
same_period_info

{'명량': ['명량', '해적: 바다로 간 산적'],
 '극한직업': ['극한직업'],
 '신과함께-죄와 벌': ['신과함께-죄와 벌', '1987'],
 '국제시장': ['국제시장'],
 '어벤져스: 엔드게임': ['어벤져스: 엔드게임', '기생충', '알라딘'],
 '겨울왕국 2': ['겨울왕국 2', '백두산'],
 '베테랑': ['베테랑', '암살'],
 '아바타': ['아바타'],
 '도둑들': ['도둑들', '광해, 왕이 된 남자'],
 '7번방의 선물': ['7번방의 선물', '베를린'],
 '알라딘': ['알라딘', '기생충', '엑시트', '스파이더맨: 파 프롬 홈'],
 '암살': ['베테랑', '암살'],
 '범죄도시2': ['범죄도시2', '탑건: 매버릭'],
 '광해, 왕이 된 남자': ['광해, 왕이 된 남자'],
 '신과함께-인과 연': ['신과함께-인과 연'],
 '택시운전사': ['택시운전사'],
 '태극기 휘날리며': ['태극기 휘날리며', '실미도'],
 '부산행': ['부산행', '밀정'],
 '변호인': ['변호인', '겨울왕국', '수상한 그녀'],
 '해운대': ['해운대', '국가대표'],
 '어벤져스: 인피니티 워': ['어벤져스: 인피니티 워'],
 '실미도': ['태극기 휘날리며', '실미도'],
 '괴물': ['괴물'],
 '아바타: 물의 길': ['아바타: 물의 길'],
 '왕의 남자': ['왕의 남자'],
 '어벤져스: 에이지 오브 울트론': ['어벤져스: 에이지 오브 울트론'],
 '인터스텔라': ['인터스텔라', '국제시장'],
 '기생충': ['알라딘', '기생충', '스파이더맨: 파 프롬 홈'],
 '겨울왕국': ['겨울왕국', '수상한 그녀', '변호인'],
 '보헤미안 랩소디': ['보헤미안 랩소디', '극한직업'],
 '검사외전': ['검사외전'],
 '엑시트': ['엑시트'],
 '설국열차': ['관상', '설국열차'],
 '관상': ['관상'],
 '아이언맨 3': ['아이언맨 3'],
 

## 결과 저장 -> json형식

In [98]:
# 딕셔너리 데이터를 JSON 문자열로 변환
my_json = json.dumps(same_period_info)

# JSON 문자열을 파일로 저장
with open('same_period_info.json', 'w') as f:
    f.write(my_json)

## 결과를 dict형으로 다시 불러오고 싶다면?

In [99]:
# JSON 파일을 읽어서 문자열로 저장
with open('same_period_info.json', 'r') as f:
    my_json = f.read()

# JSON 문자열을 딕셔너리로 변환
same_period_info1 = json.loads(my_json)

In [101]:
same_period_info1

{'명량': ['명량', '해적: 바다로 간 산적'],
 '극한직업': ['극한직업'],
 '신과함께-죄와 벌': ['신과함께-죄와 벌', '1987'],
 '국제시장': ['국제시장'],
 '어벤져스: 엔드게임': ['어벤져스: 엔드게임', '기생충', '알라딘'],
 '겨울왕국 2': ['겨울왕국 2', '백두산'],
 '베테랑': ['베테랑', '암살'],
 '아바타': ['아바타'],
 '도둑들': ['도둑들', '광해, 왕이 된 남자'],
 '7번방의 선물': ['7번방의 선물', '베를린'],
 '알라딘': ['알라딘', '기생충', '엑시트', '스파이더맨: 파 프롬 홈'],
 '암살': ['베테랑', '암살'],
 '범죄도시2': ['범죄도시2', '탑건: 매버릭'],
 '광해, 왕이 된 남자': ['광해, 왕이 된 남자'],
 '신과함께-인과 연': ['신과함께-인과 연'],
 '택시운전사': ['택시운전사'],
 '태극기 휘날리며': ['태극기 휘날리며', '실미도'],
 '부산행': ['부산행', '밀정'],
 '변호인': ['변호인', '겨울왕국', '수상한 그녀'],
 '해운대': ['해운대', '국가대표'],
 '어벤져스: 인피니티 워': ['어벤져스: 인피니티 워'],
 '실미도': ['태극기 휘날리며', '실미도'],
 '괴물': ['괴물'],
 '아바타: 물의 길': ['아바타: 물의 길'],
 '왕의 남자': ['왕의 남자'],
 '어벤져스: 에이지 오브 울트론': ['어벤져스: 에이지 오브 울트론'],
 '인터스텔라': ['인터스텔라', '국제시장'],
 '기생충': ['알라딘', '기생충', '스파이더맨: 파 프롬 홈'],
 '겨울왕국': ['겨울왕국', '수상한 그녀', '변호인'],
 '보헤미안 랩소디': ['보헤미안 랩소디', '극한직업'],
 '검사외전': ['검사외전'],
 '엑시트': ['엑시트'],
 '설국열차': ['관상', '설국열차'],
 '관상': ['관상'],
 '아이언맨 3': ['아이언맨 3'],
 

In [102]:
type(same_period_info1)  # type 확인 : dict

dict